<a href="https://colab.research.google.com/github/ykitaguchi77/GravCont_classification_2/blob/main/Crop_unilateral_eyelid_olympia_202503.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Crop_unilateral_eyelid_olympia_202503**

In [ ]:
# prompt: gdriveをマウント

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Olympia_new
zip_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/Olympia_new_dataset.zip"

# Heltel_list
Heltel_list_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/個人情報なし　コントロールと活動期.xlsx"

# YOLO11l
model_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/295+cerebhq1-9655_yolo11l.pt"

In [ ]:
# prompt: zip_pathのzipファイルを解凍

import zipfile
import os

zip_path = zip_path

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content") # Specify the extraction path

print(f"Files extracted!")


Files extracted!


In [ ]:
# prompt: # prompt: dst_dir = "content/Olympia_new/renamed_image"
# # 活動期写真： IMG_0001.JPG →A0001.JPGの名前(4桁0埋め)でdst_dir内にコピー
# # コントロール写真： 1.JPG →C0001.JPGの名前(4桁0埋め)でdst_dir内にコピー

import os
import shutil
from tqdm import tqdm

# Correct the swapped paths according to comments
active_list = "/content/Olympia_new/活動期写真"  # This should be active images
control_list = "/content/Olympia_new/コントロール写真"  # This should be control images
dst_dir = "/content/Olympia_new/renamed_image"

# Create the destination directory if it doesn't exist
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)
os.makedirs(dst_dir, exist_ok=True)

print("Processing active images...")
# Process active images with tqdm
active_files = [f for f in os.listdir(active_list) if f.lower().endswith((".jpg", ".jpeg"))]
for filename in tqdm(active_files, desc="Active images"):
    src_path = os.path.join(active_list, filename)
    name, ext = os.path.splitext(filename)
    try:
        # Check if filename is in IMG_XXXX format
        if name.startswith("IMG_"):
            num = int(name[4:])  # Extract the number part
            new_filename = f"A{num:04d}{ext.upper()}"
        else:
            # Try to convert the entire filename to a number
            num = int(name)
            new_filename = f"A{num:04d}{ext.upper()}"
    except ValueError:
        print(f"Skipping file with non-numeric name: {filename}")
        continue
    dst_path = os.path.join(dst_dir, new_filename)
    shutil.copy2(src_path, dst_path)
    #print(f"Copied {filename} to {new_filename}")

print("Processing control images...")
# Process control images with tqdm
control_files = [f for f in os.listdir(control_list) if f.lower().endswith((".jpg", ".jpeg"))]
for filename in tqdm(control_files, desc="Control images"):
    src_path = os.path.join(control_list, filename)
    name, ext = os.path.splitext(filename)
    try:
        # Try to convert the entire filename to a number
        num = int(name)
        new_filename = f"C{num:04d}{ext.upper()}"
    except ValueError:
        print(f"Skipping file with non-numeric name: {filename}")
        continue
    dst_path = os.path.join(dst_dir, new_filename)
    shutil.copy2(src_path, dst_path)

print(f"All images have been processed and saved to {dst_dir}")


Processing active images...


Active images: 100%|██████████| 1037/1037 [00:13<00:00, 75.52it/s] 


Processing control images...


Control images: 100%|██████████| 1034/1034 [00:12<00:00, 85.01it/s]

All images have been processed and saved to /content/Olympia_new/renamed_image


#**Crop eye using pre-trained YOLO11l model**

In [ ]:
!pip install ultralytics --q
!pip install wget --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.1/949.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from ultralytics import YOLO
import wget
import cv2
import matplotlib.pyplot as plt

model_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/295+cerebhq1-9655_yolo11l.pt"
model = YOLO(model_path)

# 推論を実行: YOLOの推論
image_path = r'/content/Olympia_new/renamed_image/A0001.JPG'
results = model(image_path, save=False)

for r in results:
    boxes = r.boxes  # YOLOv8の場合

    # YOLO形式出力
    for box in boxes:
        cls_id = int(box.cls[0])  # クラスID
        x_center, y_center, width, height = box.xywhn[0]

        # YOLO形式: class x_center y_center width height
        # 小数点以下6桁で出力したい場合
        print(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    #検出結果を画像に描画して表示
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(r.plot(), cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 結果の保存
    #cv2.imwrite('result.jpg', r.plot())

In [ ]:
from ultralytics import YOLO
import wget
import cv2
import numpy as np
import matplotlib.pyplot as plt

model_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/295+cerebhq1-9655_yolo11l.pt"
model = YOLO(model_path)

# クラスIDとラベルのマッピング
class_mapping = {0: "R", 1: "L"}  # クラスID 1を"L"にマッピング

# 推論を実行: YOLOの推論
image_path = r'/content/Olympia_new/renamed_image/A0001.JPG'
results = model(image_path, save=False)

# 元の画像を読み込む
original_image = cv2.imread(image_path)
img_height, img_width = original_image.shape[:2]

for r in results:
    boxes = r.boxes  # YOLOv8の場合

    # YOLO形式出力
    for i, box in enumerate(boxes):
        cls_id = int(box.cls[0])  # クラスID
        class_label = class_mapping.get(cls_id, f"Class {cls_id}")  # クラスIDに対応するラベル（R/L）

        # 正規化された座標を取得
        x_center, y_center, width, height = box.xywhn[0].cpu().numpy()

        # 正規化されたYOLO形式を出力
        print(f"{cls_id} ({class_label}) {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        # 実際のピクセル座標に変換
        x_center_px = x_center * img_width
        y_center_px = y_center * img_height
        width_px = width * img_width
        height_px = height * img_height

        # 拡張された幅を計算 (左右に25%ずつ広げる)
        expanded_width = width_px * 1.5  # 元の幅 + 50% (左右それぞれ25%)

        # 拡張された高さを計算 (拡張された幅と同じに設定)
        expanded_height = expanded_width

        # 新しい中心は元のx中心と同じ、y中心も元のまま
        new_x1 = int(x_center_px - expanded_width / 2)
        new_y1 = int(y_center_px - expanded_height / 2)  # y中心を保持
        new_x2 = int(x_center_px + expanded_width / 2)
        new_y2 = int(y_center_px + expanded_height / 2)  # y中心を保持

        # 範囲外処理のための座標計算
        pad_left = max(0, -new_x1)
        pad_top = max(0, -new_y1)
        pad_right = max(0, new_x2 - img_width)
        pad_bottom = max(0, new_y2 - img_height)

        # 画像内の実際の切り出し座標を計算
        crop_x1 = max(0, new_x1)
        crop_y1 = max(0, new_y1)
        crop_x2 = min(img_width, new_x2)
        crop_y2 = min(img_height, new_y2)

        # 切り出し
        cropped = original_image[crop_y1:crop_y2, crop_x1:crop_x2].copy()

        # パディングが必要な場合、黒で埋める
        if pad_left > 0 or pad_top > 0 or pad_right > 0 or pad_bottom > 0:
            # パディングを含む新しい画像を作成
            padded_width = crop_x2 - crop_x1 + pad_left + pad_right
            padded_height = crop_y2 - crop_y1 + pad_top + pad_bottom
            padded = np.zeros((padded_height, padded_width, 3), dtype=np.uint8)

            # 切り出した部分を黒背景に配置
            padded[pad_top:pad_top+(crop_y2-crop_y1), pad_left:pad_left+(crop_x2-crop_x1)] = cropped
            cropped = padded

        # 切り出した画像を表示（タイトルを修正）
        plt.figure(figsize=(4, 4))
        plt.title(f"{class_label} (ID: {cls_id})", fontsize=10)  # 適切なラベルを表示
        plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.tight_layout()
        plt.show()

    # 元の検出結果を表示
    result_img = cv2.cvtColor(r.plot(), cv2.COLOR_BGR2RGB)
    original_aspect_ratio = result_img.shape[1] / result_img.shape[0]

    fig_width = 6
    fig_height = fig_width / original_aspect_ratio

    plt.figure(figsize=(fig_width, fig_height))
    plt.title("Original Detection", fontsize=10)
    plt.imshow(result_img)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

###**複数枚一括処理**

Basemameに対して、右眼は_R、左眼は_Lを追加して保存

In [ ]:
from ultralytics import YOLO
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import shutil
from tqdm import tqdm

# ディレクトリ設定
orig_dir = "/content/Olympia_new/renamed_image"
dst_dir = "/content/Olympia_new/cropped_unilateral_image_25%"

# クラスIDとラベルのマッピング
class_mapping = {0: "R", 1: "L"}

# モデルの読み込み
model_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/295+cerebhq1-9655_yolo11l.pt"
model = YOLO(model_path)

# 出力ディレクトリの準備
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)  # ディレクトリが存在する場合は削除
os.makedirs(dst_dir)  # ディレクトリを新規作成

# JPGファイルのリストを取得
jpg_files = glob.glob(os.path.join(orig_dir, "*.JPG")) + glob.glob(os.path.join(orig_dir, "*.jpg"))
print(f"Total images found: {len(jpg_files)}")

# tqdmを使用してプログレスバーを表示
for image_path in tqdm(jpg_files, desc="Processing images"):
    # 元のファイル名を取得
    filename = os.path.basename(image_path)
    basename, ext = os.path.splitext(filename)

    # 画像を読み込む
    original_image = cv2.imread(image_path)
    if original_image is None:
        tqdm.write(f"Error: Could not read image {image_path}")
        continue

    img_height, img_width = original_image.shape[:2]

    # 推論を実行
    results = model(image_path, save=False)

    # 各検出結果に対して処理
    detection_count = 0
    for r in results:
        boxes = r.boxes

        for box in boxes:
            cls_id = int(box.cls[0])
            class_label = class_mapping.get(cls_id, f"Class{cls_id}")

            # 正規化された座標を取得
            x_center, y_center, width, height = box.xywhn[0].cpu().numpy()

            # 実際のピクセル座標に変換
            x_center_px = x_center * img_width
            y_center_px = y_center * img_height
            width_px = width * img_width
            height_px = height * img_height

            # 拡張された幅を計算 (左右に25%ずつ広げる)
            expanded_width = width_px * 1.5  # 元の幅 + 50% (左右それぞれ25%)

            # 拡張された高さを計算 (拡張された幅と同じに設定)
            expanded_height = expanded_width

            # 新しい中心は元のx中心と同じ、y中心も元のまま
            new_x1 = int(x_center_px - expanded_width / 2)
            new_y1 = int(y_center_px - expanded_height / 2)
            new_x2 = int(x_center_px + expanded_width / 2)
            new_y2 = int(y_center_px + expanded_height / 2)

            # 範囲外処理のための座標計算
            pad_left = max(0, -new_x1)
            pad_top = max(0, -new_y1)
            pad_right = max(0, new_x2 - img_width)
            pad_bottom = max(0, new_y2 - img_height)

            # 画像内の実際の切り出し座標を計算
            crop_x1 = max(0, new_x1)
            crop_y1 = max(0, new_y1)
            crop_x2 = min(img_width, new_x2)
            crop_y2 = min(img_height, new_y2)

            # 切り出し
            cropped = original_image[crop_y1:crop_y2, crop_x1:crop_x2].copy()

            # パディングが必要な場合、黒で埋める
            if pad_left > 0 or pad_top > 0 or pad_right > 0 or pad_bottom > 0:
                # パディングを含む新しい画像を作成
                padded_width = crop_x2 - crop_x1 + pad_left + pad_right
                padded_height = crop_y2 - crop_y1 + pad_top + pad_bottom
                padded = np.zeros((padded_height, padded_width, 3), dtype=np.uint8)

                # 切り出した部分を黒背景に配置
                padded[pad_top:pad_top+(crop_y2-crop_y1), pad_left:pad_left+(crop_x2-crop_x1)] = cropped
                cropped = padded

            # 保存するファイル名の作成 (basename_R.jpg or basename_L.jpg)
            output_filename = f"{basename}_{class_label}{ext}"
            output_path = os.path.join(dst_dir, output_filename)

            # 切り抜いた画像を保存
            cv2.imwrite(output_path, cropped)
            detection_count += 1

    # 検出されなかった場合に通知
    if detection_count == 0:
        tqdm.write(f"Warning: No eyes detected in {filename}")

print("\nProcessing completed!")
print(f"Processed {len(jpg_files)} images and saved results to {dst_dir}")

In [ ]:
import os
import shutil

# コピー元とコピー先のパス
src_dir = "/content/Olympia_new/cropped_unilateral_image_25%"
dst_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/cropped_unilateral_image_25%"

# コピー先のディレクトリが存在する場合は削除
if os.path.exists(dst_dir):
    print(f"Removing existing directory: {dst_dir}")
    shutil.rmtree(dst_dir)

# ディレクトリごとコピー
print(f"Copying {src_dir} to {dst_dir}...")
shutil.copytree(src_dir, dst_dir)
print("Copy completed successfully!")

# ファイル数を確認
file_count = len([f for f in os.listdir(dst_dir) if os.path.isfile(os.path.join(dst_dir, f))])
print(f"Total files copied: {file_count}")

Copying /content/Olympia_new/cropped_unilateral_image_25% to /content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/cropped_unilateral_image_25%...
Copy completed successfully!
Total files copied: 4142


In [ ]:
import os
import shutil
from tqdm import tqdm

# コピー元とコピー先のパス
src_dir = "/content/Olympia_new/cropped_unilateral_image_25%"
dst_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/cropped_unilateral_image_25%"

# コピー先のディレクトリが存在する場合は削除
if os.path.exists(dst_dir):
    print(f"Removing existing directory: {dst_dir}")
    shutil.rmtree(dst_dir)

# まずディレクトリを作成
os.makedirs(dst_dir)

# ファイル一覧を取得
files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]
total_files = len(files)

print(f"Copying {total_files} files from {src_dir} to {dst_dir}...")

# tqdmでファイルをコピー
for file in tqdm(files, desc="Copying files"):
    src_file = os.path.join(src_dir, file)
    dst_file = os.path.join(dst_dir, file)
    shutil.copy2(src_file, dst_file)

print("Copy completed successfully!")
print(f"Total files copied: {total_files}")

Removing existing directory: /content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/cropped_unilateral_image_25%
Copying 4142 files from /content/Olympia_new/cropped_unilateral_image_25% to /content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/cropped_unilateral_image_25%...


Copying files: 100%|██████████| 4142/4142 [01:15<00:00, 55.00it/s]

Copy completed successfully!
Total files copied: 4142


#**Hertelトレーニング用CSV対応表作成**

In [ ]:
xslx_path = "/content/Olympia_new/個人情報なし　コントロールと活動期.xlsx"